In [1]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import itertools
from networkx.algorithms import community
import operator
from itertools import islice
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_predict
from imblearn.over_sampling import SVMSMOTE
from sklearn.model_selection import cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB
import pandasql as ps
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#load dataset
df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_df_temporary4.csv")

df = df[df.Season != 2007]

df['Pace_main'] = round(df['Acceleration'] * 0.45 + df['Sprint_speed'] * 0.55, 0)
df['Shooting_main'] = round(df['Finishing'] * 0.45 + df['Long_shots'] * 0.20 + df['Penalties'] * 0.05 + \
     df['Positioning'] * 0.05 + df['Shot_power'] * 0.20 + df['Volleys'] * 0.05, 0)
df['Dribbling_main'] = round(df['Agility'] * 0.1 + df['Balance'] * 0.05 + df['Ball_control'] * 0.35 + \
     df['Dribbling'] * 0.5, 0)
df['Defending_main'] = round(df['Heading_accuracy'] * 0.1 + df['Interceptions'] * 0.2 + df['Marking'] * 0.3 + \
     df['Sliding_tackle'] * 0.3 + df['Standing_tackle'] * 0.3, 0)
df['Passing_main'] = round(df['Crossing'] * 0.2 + df['Curve'] * 0.05 + df['FK_accuracy'] * 0.05 + \
     df['Long_passing'] * 0.15 + df['Short_passing'] * 0.35 + df['Vision'] * 0.25, 0)
df['Physical_main'] = round(df['Aggression'] * 0.2 + df['Jumping'] * 0.05 + df['Stamina'] * 0.25 + \
     df['Strength'] * 0.5, 0)

/Users/roland.kovacs4ibm.com/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (13,63,120,121,122,123,124,125,126,129,130,131,133) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#load dataset
age_df = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_ws_df.xlsx")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16311 entries, 0 to 17230
Columns: 277 entries, NewName to Physical_main
dtypes: float64(208), int64(34), object(35)
memory usage: 34.6+ MB


In [5]:
df = df.drop('age', axis = 1)

In [6]:
#join tables with train df
q1 = """SELECT DF.*,
        i.Age
      
        FROM df DF 

        LEFT JOIN age_df i
        ON DF.NewName = i.NewName
        AND DF.ID = i.ID
        AND DF.Club = i.Club
        AND DF.Season = i.Season
        """

df = ps.sqldf(q1, locals())

df = df.drop_duplicates()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16311 entries, 0 to 16640
Columns: 277 entries, NewName to Age
dtypes: float64(207), int64(35), object(35)
memory usage: 34.6+ MB


In [248]:
df.head()

,NewName,MarketValue,ClubInvolved,League,MainPosition,LeagueInvolved,CorrMarketValue,ID,Name,Height_cm,...,GKCleanSheetPercent,ContractGood,ContractLeft,Pace_main,Shooting_main,Dribbling_main,Defending_main,Passing_main,Physical_main,Age
0,Bernard,10.0,Clube Atlético Mineiro,Ukrainian Premier League,Forward,Brazil,5.0,205525,Bernard,164cm,...,NaN,2017,4,90.0,71.0,85.0,42.0,80.0,41.0,20
1,Fred,5.0,Sport Club Internacional,Ukrainian Premier League,Midfielder,Brazil,2.5,209297,Fred,169cm,...,NaN,None,None,82.0,70.0,79.0,70.0,71.0,46.0,20
2,Fernando,10.0,Grêmio Foot-Ball Porto Alegrense,Ukrainian Premier League,Midfielder,Brazil,5.0,202642,Fernando,174cm,...,NaN,Free,None,78.0,66.0,69.0,92.0,74.0,78.0,21
3,W. Nem,8.0,Fluminense Football Club,Ukrainian Premier League,Forward,Brazil,4.0,203279,Wellington Nem,165cm,...,NaN,None,None,86.0,66.0,81.0,42.0,76.0,58.0,21
4,F. Ferreyra,6.0,Club Atlético Vélez Sarsfield,Ukrainian Premier League,Forward,Argentina,3.0,213432,F. Ferreyra,187cm,...,NaN,None,None,64.0,74.0,68.0,32.0,53.0,58.0,22


In [7]:
print(list(df.columns))

['NewName', 'MarketValue', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'ID', 'Name', 'Height_cm', 'Weight_kg', 'Country', 'Club', 'Joined', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes', 'Value', 'Wage', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'Contract', 'ClubInvolvedRank', 'ClubInvolvedWin', 'ClubInvolvedDraw', 'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints', 'ClubRank', 'ClubWi

In [59]:
train_df = df.copy()

In [60]:
train_df = train_df.drop(['ID', 'Name', 'Height_cm', 'Weight_kg', 'Joined', 'Value', 'Wage',\
    'Contract', 'Squad', 'Squad.1', 'seasonName', 'tournamentRegionName', 'tournamentName', 'League.1',\
        'firstName', 'lastName', 'FirstLastName', 'playerId', 'teamId', 'teamName', 'teamRegionName',\
            'playedPositions', 'positionText', 'Player', 'Pos', 'Nation',\
                'FbrefClub', 'ContractGood'], axis =1)

In [61]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16311 entries, 0 to 16640
Columns: 249 entries, NewName to Age
dtypes: float64(205), int64(34), object(10)
memory usage: 31.1+ MB


In [11]:
print(list(train_df.columns))

['NewName', 'MarketValue', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country', 'Club', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'ClubInvolvedRank', 'ClubInvolvedWin', 'ClubInvolvedDraw', 'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints', 'ClubRank', 'ClubWin', 'ClubDraw', 'ClubLoose', 'ClubGoalFor', 'ClubGoalAgainst', 'ClubPoint', 'Cl

In [12]:
percent_missing = train_df.isnull().sum() * 100 / len(train_df)
missing_value_df = pd.DataFrame({'column_name': train_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_df.head(20)

,column_name,percent_missing
GKSavePercent,GKSavePercent,97.075593
GKSaves,GKSaves,97.057201
GKCleanSheetPercent,GKCleanSheetPercent,96.952977
GKGoalAgainst,GKGoalAgainst,96.952977
GKGoalAgainstPer90,GKGoalAgainstPer90,96.952977
GKMatchPlayed,GKMatchPlayed,96.922322
GKStarts,GKStarts,96.922322
GKCleanSheet,GKCleanSheet,96.922322
GKLost,GKLost,96.922322
GKDraw,GKDraw,96.922322


In [62]:
gk_train = train_df[train_df.Position == 'GK']
outfielder_train = train_df[train_df.Position != 'GK']

Outfileders

In [14]:
print(list(set(outfielder_train.Position.values)))

['RW', 'ST', 'CDM', 'RM', 'LW', 'SW', 'CAM', 'RF', 'CF', 'RWB', 'CM', 'RB', 'LM', 'LWB', 'LF', 'LB', 'CB']


In [63]:
outfielder_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15170 entries, 0 to 16640
Columns: 249 entries, NewName to Age
dtypes: float64(205), int64(34), object(10)
memory usage: 28.9+ MB


In [44]:
gk_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1141 entries, 9 to 16600
Columns: 249 entries, NewName to Age
dtypes: float64(205), int64(34), object(10)
memory usage: 2.2+ MB


In [64]:
outfielder_train = outfielder_train.drop(['GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes',\
    'ClubInvolvedSaves', 'ClubInvolvedSaveRate', 'ClubSaves', 'ClubSaveRate', 'saveSixYardBox', 'savePenaltyArea',\
        'saveObox', 'saveTotal', 'GKMatchPlayed', 'GKStarts', 'GKMin', 'GK90s', 'GKGoalAgainst', 'GKGoalAgainstPer90',\
            'GKSaves', 'GKSavePercent', 'GKWin', 'GKDraw', 'GKLost', 'GKCleanSheet', 'GKCleanSheetPercent'], axis = 1)

In [16]:
percent_missing = outfielder_train.isnull().sum() * 100 / len(outfielder_train)
missing_value_df = pd.DataFrame({'column_name': outfielder_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_df.head(20)

,column_name,percent_missing
ShotsOnTargetPercent,ShotsOnTargetPercent,90.771259
GoalsPerShot,GoalsPerShot,90.771259
ClubInvolvedGoalPerShot,ClubInvolvedGoalPerShot,86.493078
ClubInvolvedShotsOnTargetRate,ClubInvolvedShotsOnTargetRate,86.493078
ShotsPer90,ShotsPer90,86.361239
Shots,Shots,86.361239
ClubGoalPerShot,ClubGoalPerShot,85.543837
ClubShotsOnTargetRate,ClubShotsOnTargetRate,85.543837
accurateCrossesPerGame,accurateCrossesPerGame,82.379697
keyPassPerGame_x,keyPassPerGame_x,82.379697


In [65]:
outfielder_train = outfielder_train.drop(['ShotsOnTargetPercent', 'ClubInvolvedShotsOnTargetRate', 'ClubInvolvedGoalPerShot',\
    'ShotsPer90', 'Shots', 'ClubShotsOnTargetRate', 'ClubGoalPerShot', 'ClubInvolvedShotsPer90', 'ClubInvolvedShots'], axis = 1)

In [45]:
gk_train = gk_train.drop(['ShotsOnTargetPercent', 'ClubInvolvedShotsOnTargetRate', 'ClubInvolvedGoalPerShot',\
    'ShotsPer90', 'Shots', 'ClubShotsOnTargetRate', 'ClubGoalPerShot', 'ClubInvolvedShotsPer90', 'ClubInvolvedShots'], axis = 1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1141 entries, 9 to 16600
Columns: 249 entries, NewName to Age
dtypes: float64(205), int64(34), object(10)
memory usage: 2.2+ MB


In [66]:
outf_train = outfielder_train.copy()

In [20]:
print(list(outf_train.columns))

['NewName', 'MarketValue', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country', 'Club', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'ClubInvolvedRank', 'ClubInvolvedWin', 'ClubInvolvedDraw', 'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints', 'ClubRank', 'ClubWin', 'ClubDraw', 'ClubLoose', 'ClubGoalFor', 'ClubGoalAgainst', 'ClubPoint', 'ClubInvolvedAge', 'ClubInovlvedPossession', 'ClubInvolvedAssists', 'ClubInvol

In [67]:
outf_train = outf_train.drop(['apps', 'subOn', 'minsPlayed', 'rating', 'goal_x', 'assistTotal_x', 'yellowCard_x', 'redCard_x',\
    'shotsPerGame_x', 'aerialWonPerGame', 'manOfTheMatch', 'passSuccess_x', 'tacklePerGame', 'interceptionPerGame', 'foulsPerGame',\
        'offsideWonPerGame', 'clearancePerGame', 'wasDribbledPerGame', 'outfielderBlockPerGame', 'goalOwn_x', 'keyPassPerGame_x',\
            'dribbleWonPerGame', 'foulGivenPerGame', 'offsideGivenPerGame', 'dispossessedPerGame', 'turnoverPerGame', 'totalPassesPerGame',\
                'accurateCrossesPerGame', 'accurateLongPassPerGame', 'accurateThroughBallPerGame', 'tackleTotalAttempted', 'challengeLost',\
                    'interceptionAll', 'foulCommitted', 'passCrossBlockedDefensive', 'outfielderBlockedPass',
                        'shotSixYardBox', 'shotPenaltyArea', 'shotOboxTotal', 'shotsTotal_x', 'shotOpenPlay', 'shotCounter',\
                            'shotSetPiece', 'penaltyTaken', 'shotOnTarget', 'shotOffTarget', 'shotOnPost', 'shotBlocked', 'shotRightFoot', 'shotLeftFoot',\
                                'shotHead', 'goalSixYardBox', 'goalPenaltyArea', 'goalObox', 'goalTotal_x', 'goalOwn_y', 'goalOpenPlay', 'goalCounter',\
                                    'goalSetPiece', 'penaltyScored', 'goalRightFoot', 'goalLeftFoot', 'goalHead', 'dribbleLost', 'dribbleTotal', 'duelAerialLost',\
                                        'duelAerialTotal', 'duelAerialWon', 'passTotal', 'passLongBallAccurate', 'passLongBallInaccurate', 'shortPassAccurate',\
                                            'shortPassInaccurate', 'passCornerAccurate', 'passCornerInaccurate', 'passFreekickAccurate', 'passFreekickInaccurate',\
                                                'passCrossInaccurate', 'passCrossAccurate', 'keyPassLong', 'keyPassShort', 'keyPassesTotal', 'keyPassCross', 'keyPassCorner',\
                                                    'keyPassThroughball', 'keyPassFreekick', 'keyPassThrowin', 'keyPassOther', 'assistCross', 'assistCorner', 'assistThroughball',\
                                                        'assistFreekick', 'assistThrowin', 'assistOther', 'assist',], axis=1)

In [46]:
gk_train = gk_train.drop(['apps', 'subOn', 'minsPlayed', 'rating', 'goal_x', 'assistTotal_x', 'yellowCard_x', 'redCard_x',\
    'shotsPerGame_x', 'aerialWonPerGame', 'manOfTheMatch', 'passSuccess_x', 'tacklePerGame', 'interceptionPerGame', 'foulsPerGame',\
        'offsideWonPerGame', 'clearancePerGame', 'wasDribbledPerGame', 'outfielderBlockPerGame', 'goalOwn_x', 'keyPassPerGame_x',\
            'dribbleWonPerGame', 'foulGivenPerGame', 'offsideGivenPerGame', 'dispossessedPerGame', 'turnoverPerGame', 'totalPassesPerGame',\
                'accurateCrossesPerGame', 'accurateLongPassPerGame', 'accurateThroughBallPerGame', 'tackleTotalAttempted', 'challengeLost',\
                    'interceptionAll', 'foulCommitted', 'passCrossBlockedDefensive', 'outfielderBlockedPass',
                        'shotSixYardBox', 'shotPenaltyArea', 'shotOboxTotal', 'shotsTotal_x', 'shotOpenPlay', 'shotCounter',\
                            'shotSetPiece', 'penaltyTaken', 'shotOnTarget', 'shotOffTarget', 'shotOnPost', 'shotBlocked', 'shotRightFoot', 'shotLeftFoot',\
                                'shotHead', 'goalSixYardBox', 'goalPenaltyArea', 'goalObox', 'goalTotal_x', 'goalOwn_y', 'goalOpenPlay', 'goalCounter',\
                                    'goalSetPiece', 'penaltyScored', 'goalRightFoot', 'goalLeftFoot', 'goalHead', 'dribbleLost', 'dribbleTotal', 'duelAerialLost',\
                                        'duelAerialTotal', 'duelAerialWon', 'passTotal', 'passLongBallAccurate', 'passLongBallInaccurate', 'shortPassAccurate',\
                                            'shortPassInaccurate', 'passCornerAccurate', 'passCornerInaccurate', 'passFreekickAccurate', 'passFreekickInaccurate',\
                                                'passCrossInaccurate', 'passCrossAccurate', 'keyPassLong', 'keyPassShort', 'keyPassesTotal', 'keyPassCross', 'keyPassCorner',\
                                                    'keyPassThroughball', 'keyPassFreekick', 'keyPassThrowin', 'keyPassOther', 'assistCross', 'assistCorner', 'assistThroughball',\
                                                        'assistFreekick', 'assistThrowin', 'assistOther', 'assist',], axis=1)

In [68]:
outf_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15170 entries, 0 to 16640
Columns: 119 entries, NewName to Age
dtypes: float64(80), int64(29), object(10)
memory usage: 13.9+ MB


In [23]:
percent_missing = outf_train.isnull().sum() * 100 / len(outf_train)
missing_value_df = pd.DataFrame({'column_name': outf_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_df.head(20)

,column_name,percent_missing
GoalsPerShot,GoalsPerShot,90.771259
ClubShots,ClubShots,80.507581
ClubShotsPer90,ClubShotsPer90,80.507581
ClubInovlvedPossession,ClubInovlvedPossession,78.378378
ClubPossession,ClubPossession,76.209624
Composure,Composure,61.885300
GoalPerShotOnTarget,GoalPerShotOnTarget,56.479895
ShotsOnTargetPer90,ShotsOnTargetPer90,50.672380
ShotsOnTarget,ShotsOnTarget,50.672380
AssistsPer90,AssistsPer90,47.778510


In [69]:
outf_train = outf_train.drop(['GoalsPerShot', 'ClubShotsPer90', 'ClubShots', 'ClubInovlvedPossession',\
    'ClubPossession', 'Composure'], axis = 1)

In [47]:
gk_train = gk_train.drop(['GoalsPerShot', 'ClubShotsPer90', 'ClubShots', 'ClubInovlvedPossession',\
    'ClubPossession', 'Composure'], axis = 1)

In [25]:
percent_missing = outf_train.isnull().sum() * 100 / len(outf_train)
missing_value_df = pd.DataFrame({'column_name': outf_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_df.head(20)

,column_name,percent_missing
GoalPerShotOnTarget,GoalPerShotOnTarget,56.479895
ShotsOnTargetPer90,ShotsOnTargetPer90,50.672380
ShotsOnTarget,ShotsOnTarget,50.672380
Assists,Assists,47.778510
GoalsAssistsPer90,GoalsAssistsPer90,47.778510
AssistsPer90,AssistsPer90,47.778510
NonPenaltyGoals,NonPenaltyGoals,47.738958
Starts,Starts,47.732367
RedCards,RedCards,47.732367
MatchesPlayed,MatchesPlayed,47.732367


In [26]:
print(set(outf_train.League))

{'Scottish Premiership', 'Russian Premier League', 'Italian Serie A', 'Denmark', 'Polish T-Mobile Ekstraklasa', 'Belgium', 'Norwegian Eliteserien', 'Romanian Liga I', 'Finnish Veikkausliiga', 'German 1 Bundesliga', 'Swiss Super League', 'Italian Serie B', 'Turkish Süper Lig', 'French Ligue 1', 'Austria', 'English League Championship', 'Rep Ireland Airtricity League', 'Portuguese Liga ZON SAGRES', 'French Ligue 2', 'Swedish Allsvenskan', 'Holland Eredivisie', 'Spain Primera Division', 'German 2 Bundesliga', 'Greek Super League', 'Spanish Segunda División', 'English Premier League', 'Czech Republic Gambrinus Liga', 'Ukrainian Premier League', 'Croatian Prva HNL'}


In [71]:
outf_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15170 entries, 0 to 16640
Columns: 113 entries, NewName to Age
dtypes: float64(74), int64(29), object(10)
memory usage: 13.2+ MB


In [72]:
outf_train = outf_train[outf_train.League.isin(['English Premier League', 'Italian Serie A', 'English League Championship', 'French Ligue 2',  'Spain Primera Division',\
    'Portuguese Liga ZON SAGRES','Italian Serie B', 'Spanish Segunda División', 'German 2 Bundesliga', 'German 1 Bundesliga', 'French Ligue 1',\
         'Holland Eredivisie'])]

In [73]:
outf_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11028 entries, 95 to 16640
Columns: 113 entries, NewName to Age
dtypes: float64(74), int64(29), object(10)
memory usage: 9.6+ MB


In [48]:
gk_train = gk_train[gk_train.League.isin(['English Premier League', 'Italian Serie A', 'English League Championship', 'French Ligue 2',  'Spain Primera Division',\
    'Portuguese Liga ZON SAGRES','Italian Serie B', 'Spanish Segunda División', 'German 2 Bundesliga', 'German 1 Bundesliga', 'French Ligue 1',\
         'Holland Eredivisie'])]

In [28]:
percent_missing = outf_train.isnull().sum() * 100 / len(outf_train)
missing_value_df = pd.DataFrame({'column_name': outf_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_df.head(20)

,column_name,percent_missing
GoalPerShotOnTarget,GoalPerShotOnTarget,51.759159
ShotsOnTargetPer90,ShotsOnTargetPer90,45.357272
ShotsOnTarget,ShotsOnTarget,45.357272
AssistsPer90,AssistsPer90,42.645992
Assists,Assists,42.645992
GoalsAssistsPer90,GoalsAssistsPer90,42.645992
NonPenaltyGoals,NonPenaltyGoals,42.600653
GoalsPer90,GoalsPer90,42.591585
MatchesPlayed,MatchesPlayed,42.591585
MinutesPlayed,MinutesPlayed,42.591585


In [29]:
outf_train = outf_train.drop('GoalPerShotOnTarget', axis=1)

In [49]:
gk_train = gk_train.drop('GoalPerShotOnTarget', axis=1)

In [50]:
percent_missing = gk_train.isnull().sum() * 100 / len(gk_train)
missing_value_df = pd.DataFrame({'column_name': gk_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_df.head(20)

,column_name,percent_missing
NonPenaltyGoals,NonPenaltyGoals,99.878788
MinutesPlayedPer90,MinutesPlayedPer90,99.757576
MinutesPlayed,MinutesPlayed,99.757576
Goals,Goals,99.757576
Assists,Assists,99.757576
Starts,Starts,99.757576
YellowCards,YellowCards,99.757576
RedCards,RedCards,99.757576
GoalsPer90,GoalsPer90,99.757576
AssistsPer90,AssistsPer90,99.757576


In [51]:
gk_train = gk_train.drop(['NonPenaltyGoals', 'MinutesPlayedPer90', 'MinutesPlayed', 'Goals', 'Assists',\
    'Starts', 'YellowCards', 'RedCards', 'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'MatchesPlayed', 'ShotsOnTarget',\
        'ShotsOnTargetPer90', 'saveTotal', 'saveObox', 'savePenaltyArea', 'saveSixYardBox', 'GKSavePercent', 'GKSaves'], axis=1)

In [52]:
percent_missing = gk_train.isnull().sum() * 100 / len(gk_train)
missing_value_df = pd.DataFrame({'column_name': gk_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
missing_value_df.head(20)

,column_name,percent_missing
GKCleanSheetPercent,GKCleanSheetPercent,50.909091
GKGoalAgainstPer90,GKGoalAgainstPer90,50.666667
GKGoalAgainst,GKGoalAgainst,50.666667
GKMatchPlayed,GKMatchPlayed,50.424242
GKMin,GKMin,50.424242
GKCleanSheet,GKCleanSheet,50.424242
GKLost,GKLost,50.424242
GKDraw,GKDraw,50.424242
GKWin,GKWin,50.424242
GKStarts,GKStarts,50.424242


In [53]:
gk_train = gk_train.drop(['GKCleanSheetPercent', 'GKGoalAgainstPer90', 'GKGoalAgainst', 'GKMatchPlayed', 'GKMin',\
    'GKCleanSheet', 'GKLost', 'GKDraw', 'GKWin', 'GKStarts', 'GK90s'], axis=1)

In [55]:
print(list(gk_train.columns))

['NewName', 'MarketValue', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country', 'Club', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'ClubInvolvedRank', 'ClubInvolvedWin', 'ClubInvolvedDraw', 'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints', 'ClubRank', 'ClubWin', 'ClubDraw', 'ClubLoose', 'ClubGoalFor', 'ClubGoalAgainst', 'ClubPoint', 'ClubInvolvedAge

In [56]:
gk_train = gk_train.drop(['Crossing', 'Finishing', 'Heading_accuracy', 'Ball_control', 'Acceleration', 'Sprint_speed',\
    'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Shot_power', 'Strength', 'Long_shots', 'Aggression', 'Interceptions',\
        'Vision', 'Penalties', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'Pace_main', 'Shooting_main', 'Dribbling_main',\
            'Defending_main', 'Passing_main', 'Physical_main',], axis=1)

In [57]:
gk_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825 entries, 140 to 16500
Data columns (total 81 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   NewName                          825 non-null    object 
 1   MarketValue                      825 non-null    float64
 2   ClubInvolved                     825 non-null    object 
 3   League                           825 non-null    object 
 4   MainPosition                     825 non-null    object 
 5   LeagueInvolved                   825 non-null    object 
 6   CorrMarketValue                  825 non-null    float64
 7   Country                          825 non-null    object 
 8   Club                             825 non-null    object 
 9   Position                         825 non-null    object 
 10  Foot                             825 non-null    object 
 11  IntReputation                    825 non-null    int64  
 12  Short_passing     

In [30]:
print(list(outf_train.columns))

['NewName', 'MarketValue', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country', 'Club', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'ClubInvolvedRank', 'ClubInvolvedWin', 'ClubInvolvedDraw', 'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints', 'ClubRank', 'ClubWin', 'ClubDraw', 'ClubLoose', 'ClubGoalFor', 'ClubGoalAgainst', 'ClubPoint', 'ClubInvolvedAge', 'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellow

In [31]:
outf_df = outf_train[['NewName', 'ClubInvolved','Club']]

In [32]:
outf_train['ClubInvolvedPlayed'] = outf_train['ClubInvolvedWin'] + outf_train['ClubInvolvedDraw'] + outf_train['ClubInvolvedLoose']
outf_train['ClubInvolvedWinPerMatch'] = outf_train['ClubInvolvedWin'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedDrawPerMatch'] = outf_train['ClubInvolvedDraw'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedLoosePerMatch'] = outf_train['ClubInvolvedLoose'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedGoalForPerMatch'] = outf_train['ClubInvolvedGoalFor'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedGoalAgainstPerMatch'] = outf_train['ClubInvolvedGoalAgainst'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedPointsPerMatch'] = outf_train['ClubInvolvedPoints'] / outf_train['ClubInvolvedPlayed']

outf_train['ClubPlayed'] = outf_train['ClubWin'] + outf_train['ClubDraw'] + outf_train['ClubLoose']
outf_train['ClubWinPerMatch'] = outf_train['ClubWin'] / outf_train['ClubPlayed']
outf_train['ClubDrawPerMatch'] = outf_train['ClubDraw'] / outf_train['ClubPlayed']
outf_train['ClubLoosePerMatch'] = outf_train['ClubLoose'] / outf_train['ClubPlayed']
outf_train['ClubGoalForPerMatch'] = outf_train['ClubGoalFor'] / outf_train['ClubPlayed']
outf_train['ClubGoalAgainstPerMatch'] = outf_train['ClubGoalAgainst'] / outf_train['ClubPlayed']
outf_train['ClubPointsPerMatch'] = outf_train['ClubPoint'] / outf_train['ClubPlayed']

outf_train['ClubInvolvedAssistsPerMatch'] = outf_train['ClubInvolvedAssists'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedNonPenaltyGoalPerMatch'] = outf_train['ClubInvolvedNonPenaltyGoal'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedYellowCardPerMatch'] = outf_train['ClubInvolvedYellowCard'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedRedCardPerMatch'] = outf_train['ClubInvolvedRedCard'] / outf_train['ClubInvolvedPlayed']

outf_train['ClubAssistsPerMatch'] = outf_train['ClubAssists'] / outf_train['ClubPlayed']
outf_train['ClubNonPenaltyGoalPerMatch'] = outf_train['ClubNonPenaltyGoal'] / outf_train['ClubPlayed']
outf_train['ClubYellowCardPerMatch'] = outf_train['ClubYellowCard'] / outf_train['ClubPlayed']
outf_train['ClubRedCardPerMatch'] = outf_train['ClubRedCard'] / outf_train['ClubPlayed']

outf_train['ClubInvolvedNumPlayersUsedPerMatch'] = outf_train['ClubInvolvedNumPlayersUsed'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedShotsOnTargetPerMatch'] = outf_train['ClubInvolvedShotsOnTarget'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedGoalPerShotOnTargetPerMatch'] = outf_train['ClubInvolvedGoalPerShotOnTarget'] / outf_train['ClubInvolvedPlayed']

outf_train['ClubNumPlayersUsedPerMatch'] = outf_train['ClubNumPlayersUsed'] / outf_train['ClubPlayed']
outf_train['ClubShotsOnTargetPerMatch'] = outf_train['ClubShotsOnTarget'] / outf_train['ClubPlayed']
outf_train['ClubGoalPerShotOnTargetPerMatch'] = outf_train['ClubGoalPerShotOnTarget'] / outf_train['ClubPlayed']

outf_train['ClubInvolvedShotOnTargetAgainstUsedPerMatch'] = outf_train['ClubInvolvedShotOnTargetAgainst'] / outf_train['ClubInvolvedPlayed']
outf_train['ClubInvolvedCleanSheetsPerMatch'] = outf_train['ClubInvolvedCleanSheets'] / outf_train['ClubInvolvedPlayed']

outf_train['ClubShotOnTargetAgainstUsedPerMatch'] = outf_train['ClubShotOnTargetAgainst'] / outf_train['ClubPlayed']
outf_train['ClubCleanSheetsPerMatch'] = outf_train['ClubCleanSheets'] / outf_train['ClubPlayed']

outf_train['MatchesPlayedPerClubMatches'] = outf_train['MatchesPlayed'] / outf_train['ClubInvolvedPlayed']
outf_train['StartsPerClubMatches'] = outf_train['Starts'] / outf_train['ClubInvolvedPlayed']
outf_train['NonPenaltyGoalsPerMinutes'] = outf_train['MatchesPlayed'] / outf_train['ClubInvolvedPlayed']
outf_train['GoalsPerMinutesPlayed'] = outf_train['Goals'] / outf_train['MinutesPlayed']
outf_train['AssistsPerMinutesPlayed'] = outf_train['Assists'] / outf_train['MinutesPlayed']
outf_train['NonPenaltyGoalsPerMinutesPlayed'] = outf_train['NonPenaltyGoals'] / outf_train['MinutesPlayed']
outf_train['YellowCardsPerMinutesPlayed'] = outf_train['YellowCards'] / outf_train['MinutesPlayed']
outf_train['RedCardsPerMinutesPlayed'] = outf_train['RedCards'] / outf_train['MinutesPlayed']
outf_train['ShotsOnTargetPerMinutesPlayed'] = outf_train['ShotsOnTarget'] / outf_train['MinutesPlayed']

In [58]:
gk_train['ClubInvolvedPlayed'] = gk_train['ClubInvolvedWin'] + gk_train['ClubInvolvedDraw'] + gk_train['ClubInvolvedLoose']
gk_train['ClubInvolvedWinPerMatch'] = gk_train['ClubInvolvedWin'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedDrawPerMatch'] = gk_train['ClubInvolvedDraw'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedLoosePerMatch'] = gk_train['ClubInvolvedLoose'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedGoalForPerMatch'] = gk_train['ClubInvolvedGoalFor'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedGoalAgainstPerMatch'] = gk_train['ClubInvolvedGoalAgainst'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedPointsPerMatch'] = gk_train['ClubInvolvedPoints'] / gk_train['ClubInvolvedPlayed']

gk_train['ClubPlayed'] = gk_train['ClubWin'] + gk_train['ClubDraw'] + gk_train['ClubLoose']
gk_train['ClubWinPerMatch'] = gk_train['ClubWin'] / gk_train['ClubPlayed']
gk_train['ClubDrawPerMatch'] = gk_train['ClubDraw'] / gk_train['ClubPlayed']
gk_train['ClubLoosePerMatch'] = gk_train['ClubLoose'] / gk_train['ClubPlayed']
gk_train['ClubGoalForPerMatch'] = gk_train['ClubGoalFor'] / gk_train['ClubPlayed']
gk_train['ClubGoalAgainstPerMatch'] = gk_train['ClubGoalAgainst'] / gk_train['ClubPlayed']
gk_train['ClubPointsPerMatch'] = gk_train['ClubPoint'] / gk_train['ClubPlayed']

gk_train['ClubInvolvedAssistsPerMatch'] = gk_train['ClubInvolvedAssists'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedNonPenaltyGoalPerMatch'] = gk_train['ClubInvolvedNonPenaltyGoal'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedYellowCardPerMatch'] = gk_train['ClubInvolvedYellowCard'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedRedCardPerMatch'] = gk_train['ClubInvolvedRedCard'] / gk_train['ClubInvolvedPlayed']

gk_train['ClubAssistsPerMatch'] = gk_train['ClubAssists'] / gk_train['ClubPlayed']
gk_train['ClubNonPenaltyGoalPerMatch'] = gk_train['ClubNonPenaltyGoal'] / gk_train['ClubPlayed']
gk_train['ClubYellowCardPerMatch'] = gk_train['ClubYellowCard'] / gk_train['ClubPlayed']
gk_train['ClubRedCardPerMatch'] = gk_train['ClubRedCard'] / gk_train['ClubPlayed']

gk_train['ClubInvolvedNumPlayersUsedPerMatch'] = gk_train['ClubInvolvedNumPlayersUsed'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedShotsOnTargetPerMatch'] = gk_train['ClubInvolvedShotsOnTarget'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedGoalPerShotOnTargetPerMatch'] = gk_train['ClubInvolvedGoalPerShotOnTarget'] / gk_train['ClubInvolvedPlayed']

gk_train['ClubNumPlayersUsedPerMatch'] = gk_train['ClubNumPlayersUsed'] / gk_train['ClubPlayed']
gk_train['ClubShotsOnTargetPerMatch'] = gk_train['ClubShotsOnTarget'] / gk_train['ClubPlayed']
gk_train['ClubGoalPerShotOnTargetPerMatch'] = gk_train['ClubGoalPerShotOnTarget'] / gk_train['ClubPlayed']

gk_train['ClubInvolvedShotOnTargetAgainstUsedPerMatch'] = gk_train['ClubInvolvedShotOnTargetAgainst'] / gk_train['ClubInvolvedPlayed']
gk_train['ClubInvolvedCleanSheetsPerMatch'] = gk_train['ClubInvolvedCleanSheets'] / gk_train['ClubInvolvedPlayed']

gk_train['ClubShotOnTargetAgainstUsedPerMatch'] = gk_train['ClubShotOnTargetAgainst'] / gk_train['ClubPlayed']
gk_train['ClubCleanSheetsPerMatch'] = gk_train['ClubCleanSheets'] / gk_train['ClubPlayed']

gk_train['MatchesPlayedPerClubMatches'] = gk_train['MatchesPlayed'] / gk_train['ClubInvolvedPlayed']
gk_train['StartsPerClubMatches'] = gk_train['Starts'] / gk_train['ClubInvolvedPlayed']
gk_train['NonPenaltyGoalsPerMinutes'] = gk_train['MatchesPlayed'] / gk_train['ClubInvolvedPlayed']
gk_train['GoalsPerMinutesPlayed'] = gk_train['Goals'] / gk_train['MinutesPlayed']
gk_train['AssistsPerMinutesPlayed'] = gk_train['Assists'] / gk_train['MinutesPlayed']
gk_train['NonPenaltyGoalsPerMinutesPlayed'] = gk_train['NonPenaltyGoals'] / gk_train['MinutesPlayed']
gk_train['YellowCardsPerMinutesPlayed'] = gk_train['YellowCards'] / gk_train['MinutesPlayed']
gk_train['RedCardsPerMinutesPlayed'] = gk_train['RedCards'] / gk_train['MinutesPlayed']
gk_train['ShotsOnTargetPerMinutesPlayed'] = gk_train['ShotsOnTarget'] / gk_train['MinutesPlayed']

KeyError: 'MatchesPlayed'

In [33]:
outf_train = outf_train.drop(['ClubInvolvedPlayed', 'ClubInvolvedWin', 'ClubInvolvedDraw',\
     'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints',\
          'ClubPlayed', 'ClubWin', 'ClubDraw', 'ClubLoose', 'ClubGoalFor', 'ClubGoalAgainst', 'ClubPoint',\
               'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellowCard', 'ClubInvolvedRedCard',\
                    'ClubAssists', 'ClubNonPenaltyGoal', 'ClubYellowCard', 'ClubRedCard', 'ClubInvolvedNumPlayersUsed',\
                         'ClubInvolvedShotsOnTarget', 'ClubInvolvedGoalPerShotOnTarget', 'ClubNumPlayersUsed', 'ClubShotsOnTarget',\
                         'ClubGoalPerShotOnTarget', 'ClubInvolvedShotOnTargetAgainst', 'ClubInvolvedCleanSheets', 'ClubShotOnTargetAgainst',\
                              'ClubCleanSheets', 'MatchesPlayed', 'Starts', 'MatchesPlayed', 'Goals', 'Assists', 'NonPenaltyGoals', 'YellowCards',\
                                   'RedCards', 'ShotsOnTarget',  'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'ShotsOnTargetPer90'], axis=1)

In [34]:
print(list(outf_train.columns))

['NewName', 'MarketValue', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country', 'Club', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'ClubInvolvedRank', 'ClubRank', 'ClubInvolvedAge', 'ClubAge', 'ClubInvolvedShotsOnTargetPer90', 'ClubShotsOnTargetPer90', 'ClubInvolvedGoalAgainstPer90', 'ClubInvolvedCleanSheetsRate', 'ClubGoalAgainstPer90', 'ClubCleanSheetsRate', 'MinutesPlayed', 'MinutesPlayedPer90', 'ContractLeft', 'Pace_main', 'Shooting_main', 'Dribbling_main', 'De

In [35]:
outf_train_df = outf_train.copy()


In [36]:
outf_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11028 entries, 95 to 16640
Columns: 109 entries, NewName to ShotsOnTargetPerMinutesPlayed
dtypes: float64(70), int64(29), object(10)
memory usage: 9.3+ MB


In [42]:
outf_train_df.Fee.describe()

count    8859.000000
mean        3.977810
std         9.107186
min         0.000000
25%         0.050000
50%         0.890000
75%         3.800000
max       222.000000
Name: Fee, dtype: float64

In [37]:
outf_train_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/TRAIN2.csv")

In [38]:
league_outf_train_df = outf_train_df.drop(['NewName', 'MarketValue','ClubInvolved', 'Club', 'ClubRank', 'ClubAge', 'ClubShotsOnTargetPer90',\
    'ClubGoalAgainstPer90', 'ClubCleanSheetsRate', 'ClubWinPerMatch', 'ClubDrawPerMatch', 'ClubLoosePerMatch', 'ClubGoalForPerMatch', 'ClubGoalAgainstPerMatch',\
        'ClubPointsPerMatch', 'ClubAssistsPerMatch', 'ClubNonPenaltyGoalPerMatch', 'ClubYellowCardPerMatch', 'ClubRedCardPerMatch','ClubNumPlayersUsedPerMatch',\
            'ClubShotsOnTargetPerMatch', 'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch', 'ClubCleanSheetsPerMatch'], axis = 1)
club_outf_train_df = outf_train_df.drop(['NewName', 'MarketValue'], axis = 1)

In [39]:
print(list(league_outf_train_df.columns))

['League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'ClubInvolvedRank', 'ClubInvolvedAge', 'ClubInvolvedShotsOnTargetPer90', 'ClubInvolvedGoalAgainstPer90', 'ClubInvolvedCleanSheetsRate', 'MinutesPlayed', 'MinutesPlayedPer90', 'ContractLeft', 'Pace_main', 'Shooting_main', 'Dribbling_main', 'Defending_main', 'Passing_main', 'Physical_main', 'Age', 'ClubInvolvedWinPerMatch', 'ClubInvolvedDrawPerMatch', 'ClubInvolvedLoosePerMatch', 'ClubIn

In [37]:
league_outf_train_df['target'] = 0
league_outf_train_df.loc[league_outf_train_df.League == 'Spain Primera Division', 'target'] = 1

In [316]:
club_outf_train_df['target'] = 0
club_outf_train_df.loc[club_outf_train_df.Club == 'Chelsea', 'target'] = 1

In [38]:
league_outf_train_df['target'].value_counts()

0    10034
1      994
Name: target, dtype: int64

In [317]:
club_outf_train_df['target'].value_counts()

0    15108
1       62
Name: target, dtype: int64

In [319]:
club_outf_train_df = club_outf_train_df.drop(['ClubInvolved', 'Club', 'League'], axis=1)

In [329]:
club_outf_train_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_chelsea.csv")

In [330]:
club_outf_train_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_chelsea.xlsx")

In [328]:
club_outf_train_df.head()

,MainPosition,LeagueInvolved,CorrMarketValue,Country,Position,Foot,IntReputation,Crossing,Finishing,Heading_accuracy,...,MatchesPlayedPerClubMatches,StartsPerClubMatches,NonPenaltyGoalsPerMinutes,GoalsPerMinutesPlayed,AssistsPerMinutesPlayed,NonPenaltyGoalsPerMinutesPlayed,YellowCardsPerMinutesPlayed,RedCardsPerMinutesPlayed,ShotsOnTargetPerMinutesPlayed,target
0,Forward,Brazil,5.0,Brazil,RW,Right,1,78,73,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Midfielder,Brazil,2.5,Brazil,CAM,Left,1,66,74,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Midfielder,Brazil,5.0,Brazil,CDM,Right,1,62,65,62,...,NaN,NaN,NaN,0.000521,0.000521,0.000521,0.004685,0.0,0.002082,0
3,Forward,Brazil,4.0,Brazil,LM,Left,1,72,69,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Forward,Argentina,3.0,Argentina,ST,Right,1,48,77,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [39]:
league_outf_train_df = league_outf_train_df.drop('League', axis = 1)

In [40]:
#set feature list and target
feature_list = list(league_outf_train_df.columns.values)
feature_list.remove('target')
target = 'target'

# define dataset
X = league_outf_train_df[feature_list]
y = league_outf_train_df[target]

In [348]:
#set feature list and target
feature_list = list(club_outf_train_df.columns.values)
feature_list.remove('target')
target = 'target'

# define dataset
X = club_outf_train_df[feature_list]
y = club_outf_train_df[target]

ValueError: list.remove(x): x not in list

In [41]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

In [42]:
#Impute missing values
imputer = IterativeImputer()
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [43]:
#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

In [44]:
# define the model
model = XGBClassifier(n_estimators=50)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

[18:27:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:27:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:27:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:27:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[18:28:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:28:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:28:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:28:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

...

[18:29:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:29:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.war

[18:29:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:29:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 29.59111578464508
score_time 0.1670628786087036
test_acc 0.9299958206705684
test_recall 0.39834848484848484
test_auc 0.9188336492394121
test_precison 0.6988463529852529
test_f1 0.5058436687513967


In [45]:
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 29.59111578464508
score_time 0.1670628786087036
test_acc 0.9299958206705684
test_recall 0.39834848484848484
test_auc 0.9188336492394121
test_precison 0.6988463529852529
test_f1 0.5058436687513967


In [327]:
print(list(X_imp_st.columns))

['MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'ClubInvolvedRank', 'ClubRank', 'ClubInvolvedAge', 'ClubAge', 'ClubInvolvedShotsOnTargetPer90', 'ClubShotsOnTargetPer90', 'ClubInvolvedGoalAgainstPer90', 'ClubInvolvedCleanSheetsRate', 'ClubGoalAgainstPer90', 'ClubCleanSheetsRate', 'MinutesPlayed', 'MinutesPlayedPer90', 'ContractLeft', 'Pace_main', 'Shooting_main', 'Dribbling_main', 'Defending_main', 'Passing_main', 'Physical_main', 'Age', 'ClubInvolvedWinPerMat

Predict


In [298]:
predict_df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp3.csv")
predict_df = predict_df.drop('Unnamed: 0', axis = 1)

In [182]:
predict_df.head()

,Name,Age,Nationality,Pos,MarketValue,LeagueInvolved,Crossing,Finishing,Heading_accuracy,Short_passing,...,GK90s,GKGoalAgainst,GKGoalAgainstPer90,GKSaves,GKSavePercent,GKWin,GKDraw,GKLost,GKCleanSheet,GKCleanSheetPercent
0,L. Messi,33,Argentina,"FW,MF",103.5,Spain Primera Division,85,95,70,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cristiano Ronaldo,35,Portugal,FW,63.0,Italian Serie A,84,95,90,82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,J. Oblak,27,Slovenia,NaN,120.0,Spain Primera Division,13,11,15,43,...,38.0,25.0,0.66,97.0,80.0,26.0,8.0,4.0,18.0,47.4
3,K. De Bruyne,29,Belgium,MF,127.5,English Premier League,94,82,55,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Neymar Jr,28,Brazil,NaN,132.0,French Ligue 1,85,87,62,87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [299]:
predict_df['Pace_main'] = round(predict_df['Acceleration'] * 0.45 + predict_df['Sprint_speed'] * 0.55, 0)
predict_df['Shooting_main'] = round(predict_df['Finishing'] * 0.45 + predict_df['Long_shots'] * 0.20 + predict_df['Penalties'] * 0.05 + \
     predict_df['Positioning'] * 0.05 + predict_df['Shot_power'] * 0.20 + predict_df['Volleys'] * 0.05, 0)
predict_df['Dribbling_main'] = round(predict_df['Agility'] * 0.1 + predict_df['Balance'] * 0.05 + predict_df['Ball_control'] * 0.35 + \
     predict_df['Dribbling'] * 0.5, 0)
predict_df['Defending_main'] = round(predict_df['Heading_accuracy'] * 0.1 + predict_df['Interceptions'] * 0.2 + predict_df['Marking'] * 0.3 + \
     predict_df['Sliding_tackle'] * 0.3 + predict_df['Standing_tackle'] * 0.3, 0)
predict_df['Passing_main'] = round(predict_df['Crossing'] * 0.2 + predict_df['Curve'] * 0.05 + predict_df['FK_accuracy'] * 0.05 + \
     predict_df['Long_passing'] * 0.15 + predict_df['Short_passing'] * 0.35 + predict_df['Vision'] * 0.25, 0)
predict_df['Physical_main'] = round(predict_df['Aggression'] * 0.2 + predict_df['Jumping'] * 0.05 + predict_df['Stamina'] * 0.25 + \
     predict_df['Strength'] * 0.5, 0)

In [300]:
gk_predict = predict_df[predict_df.Position == 'GK']
outfielder_predict = predict_df[predict_df.Position != 'GK']

In [301]:
outfielder_predict = outfielder_predict.drop(['GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes',\
    'ClubInvolvedSaves', 'ClubInvolvedSaveRate', 'GKMatchPlayed', 'GKStarts', 'GKMin', 'GK90s', 'GKGoalAgainst', 'GKGoalAgainstPer90',\
        'GKSaves', 'GKSavePercent', 'GKWin', 'GKDraw', 'GKLost', 'GKCleanSheet', 'GKCleanSheetPercent'], axis = 1)

In [302]:
outfielder_predict = outfielder_predict.drop(['ShotsOnTargetPercent', 'ClubInvolvedShotsOnTargetRate', 'ClubInvolvedGoalPerShot',\
    'ShotsPer90', 'Shots', 'ClubInvolvedShotsPer90', 'ClubInvolvedShots'], axis = 1)

In [303]:
outfielder_predict = outfielder_predict.drop(['GoalsPerShot', 'ClubInovlvedPossession'], axis = 1)

In [304]:
outfielder_predict['ClubInvolvedPlayed'] = outfielder_predict['ClubInvolvedWin'] + outfielder_predict['ClubInvolvedDraw'] + outfielder_predict['ClubInvolvedLoose']
outfielder_predict['ClubInvolvedWinPerMatch'] = outfielder_predict['ClubInvolvedWin'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedDrawPerMatch'] = outfielder_predict['ClubInvolvedDraw'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedLoosePerMatch'] = outfielder_predict['ClubInvolvedLoose'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalForPerMatch'] = outfielder_predict['ClubInvolvedGoalFor'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalAgainstPerMatch'] = outfielder_predict['ClubInvolvedGoalAgainst'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedPointsPerMatch'] = outfielder_predict['ClubInvolvedPoints'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedAssistsPerMatch'] = outfielder_predict['ClubInvolvedAssists'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedNonPenaltyGoalPerMatch'] = outfielder_predict['ClubInvolvedNonPenaltyGoal'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedYellowCardPerMatch'] = outfielder_predict['ClubInvolvedYellowCard'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedRedCardPerMatch'] = outfielder_predict['ClubInvolvedRedCard'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedNumPlayersUsedPerMatch'] = outfielder_predict['ClubInvolvedNumPlayersUsed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedShotsOnTargetPerMatch'] = outfielder_predict['ClubInvolvedShotsOnTarget'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalPerShotOnTargetPerMatch'] = outfielder_predict['ClubInvolvedGoalPerShotOnTarget'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedShotOnTargetAgainstUsedPerMatch'] = outfielder_predict['ClubInvolvedShotOnTargetAgainst'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedCleanSheetsPerMatch'] = outfielder_predict['ClubInvolvedCleanSheets'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['MatchesPlayedPerClubMatches'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['StartsPerClubMatches'] = outfielder_predict['Starts'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutes'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['GoalsPerMinutesPlayed'] = outfielder_predict['Goals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['AssistsPerMinutesPlayed'] = outfielder_predict['Assists'] / outfielder_predict['MinutesPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutesPlayed'] = outfielder_predict['NonPenaltyGoals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['YellowCardsPerMinutesPlayed'] = outfielder_predict['YellowCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['RedCardsPerMinutesPlayed'] = outfielder_predict['RedCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['ShotsOnTargetPerMinutesPlayed'] = outfielder_predict['ShotsOnTarget'] / outfielder_predict['MinutesPlayed']

In [305]:
outfielder_predict = outfielder_predict.drop(['ClubInvolvedPlayed', 'ClubInvolvedWin', 'ClubInvolvedDraw',\
     'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints',\
               'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellowCard', 'ClubInvolvedRedCard',\
                     'ClubInvolvedNumPlayersUsed',\
                         'ClubInvolvedShotsOnTarget', 'ClubInvolvedGoalPerShotOnTarget', 'ClubInvolvedShotOnTargetAgainst', 'ClubInvolvedCleanSheets',\
                              'MatchesPlayed', 'Starts', 'MatchesPlayed', 'Goals', 'Assists', 'NonPenaltyGoals', 'YellowCards',\
                                   'RedCards', 'ShotsOnTarget',  'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'ShotsOnTargetPer90'], axis=1)

In [190]:
print(list(X_imp_st.columns))

['MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'Country', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'ClubInvolvedRank', 'ClubInvolvedAge', 'ClubInvolvedShotsOnTargetPer90', 'ClubInvolvedGoalAgainstPer90', 'ClubInvolvedCleanSheetsRate', 'MinutesPlayed', 'MinutesPlayedPer90', 'ContractLeft', 'Pace_main', 'Shooting_main', 'Dribbling_main', 'Defending_main', 'Passing_main', 'Physical_main', 'ClubInvolvedWinPerMatch', 'ClubInvolvedDrawPerMatch', 'ClubInvolvedLoosePerMatch', 'ClubInvolvedGoalForPerMatch', 'ClubInvol

In [306]:
outfielder_predict['CorrMarketValue'] = outfielder_predict['NewMarketValue']

In [307]:
result_df = outfielder_predict[['Name', 'ClubInvolved', 'Pos',  'ID', 'Nationality']]

In [308]:
outfielder_predict = outfielder_predict.drop([ 'Name', 'Season', 'ClubInvolved', 'ContractGood', 'Player',\
    'Nation', 'FbrefClub', 'GoalPerShotOnTarget', 'Pos', 'MarketValue', 'Composure', 'ID', 'Joined', 'NewMarketValue', 'Nationality'], axis=1)

In [309]:
#encode
outfielder_predict_cbe = cbe_encoder.transform(outfielder_predict)

In [310]:
#impute
outfielder_predict_cbe_imp = imputer.transform(outfielder_predict_cbe)
outfielder_predict_cbe_imp = pd.DataFrame(data = outfielder_predict_cbe_imp, columns = outfielder_predict_cbe.columns)

In [311]:
#standardization
outfielder_predict_imp_st = scaler.transform(outfielder_predict_cbe_imp)
outfielder_predict_imp_st = pd.DataFrame(data = outfielder_predict_imp_st, columns = outfielder_predict_cbe_imp.columns)

In [313]:
#predict 
model.fit(X_imp_st , y)
result_df['confidence'] = model.predict_proba(outfielder_predict_imp_st)[:, 1]

[19:07:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [314]:
result_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/result.xlsx")